In [383]:
import tensorflow as tf 
import csv 
import numpy as np
import random
import pandas as pd
import mitdeeplearning as mdl
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten


import datetime

In [384]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [385]:
rm -rf ./logs/

In [386]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [387]:
def file_select():  
    global accelometer
    global gyroscope
    global output_data
    global speed
    global df

    num = random.randint(3,9)
    df = pd.read_csv('/Users/amitaflalo/Desktop/deepnav/data/train/locationData ' + str(num) + '.csv', header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    output_data = df.iloc[:, 9:10] 
    speed = df.iloc[:, 9:10]

In [388]:
def file_select_test():  
    global accelometer
    global gyroscope
    global output_data
    global speed
    global df

    df = pd.read_csv('/Users/amitaflalo/Desktop/deepnav/data/test/locationData'  + '.csv', header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    output_data = df.iloc[:, 9:10] 
    speed = df.iloc[:, 9:10]

In [389]:
def get_batch(accelometer, gyroscope, output_data, speed, batch_size):
    
    idx = random.randint(0, ((len(df) - (batch_size + 1) * 10) // 10))

    #input to Network
    ##########################################################################################

    input_batch_accelometer = [accelometer.iloc[i * 10 : i * 10 +10,:] for i in range(idx,idx + batch_size)]
    input_batch_gyroscope = [gyroscope.iloc[i * 10 : i * 10 +10,:] for i in range(idx,idx + batch_size)]

    a = np.array(input_batch_accelometer)
    b = np.array(input_batch_gyroscope)
    
    x_out = np.concatenate([a,b], axis= 2)

    #ground truth[azimut diff, acceleraion(m/s)]
    ########################################################################################## 

    
    out = [output_data.iloc[i * 10,:] for i in range(idx + 1,idx + batch_size + 1)]
    y = np.array(out, dtype='float64')

    x_out = np.expand_dims(x_out, axis = 3)

    return x_out, y

In [390]:
x, y = get_batch(accelometer, gyroscope, output_data, speed, 1)

In [391]:
x.shape


(1, 10, 6, 1)

In [392]:
y = Conv2D(16, (1,3), strides=(1,3), activation='relu', input_shape = (10,6,1))(x)
y = Conv2D(32, (2,2), strides=(1,1), activation='relu')(y)

# y = MaxPool2D(pool_size=(2,2))(y)

In [393]:
print(y.shape)

(1, 9, 1, 32)


In [394]:
def build_model():
  model = tf.keras.Sequential([
    Conv2D(16, (1,3), strides=(1,3), activation='relu', input_shape = (10,6,1)),
    Conv2D(32, (2,2), strides=(1,1), activation='relu'),
    Conv2D(32, (2,1), strides=(2,1), activation='relu'),
    Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
  ])

  return model



In [395]:
def compute_loss(labels, logits):
  mse = tf.keras.losses.MeanAbsoluteError()
  loss = mse(labels, logits)
  return loss
# mse = tf.keras.losses.MeanSquareError()

In [396]:
def precent(labels, logits):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    loss = mae(labels, logits)
      
    return loss

In [397]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

In [398]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations= 2000  # Increase this to train longer
batch_size = 3  # Experiment between 1 and 64
learning_rate = 1e-3  # Experiment between 1e-5 and 1e-1

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [399]:

model = build_model()
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
      y_hat = model(x) 
      loss = compute_loss(y, y_hat)

  # Now, compute the gradients 
  grads = tape.gradient(loss, model.trainable_variables) 
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  #metrics
  train_loss(loss)

@tf.function
def test_step(x, y): 
    y_hat = model(x) 
    # loss = compute_loss(y, y_hat)
    loss = precent(y, y_hat)
    
    test_loss(loss)

In [400]:
##################
# Begin training!#
##################

for iter in range(num_training_iterations):

  # Grab a batch and propagate it through the network
  file_select()
  x_batch, y_batch = get_batch(accelometer, gyroscope,speed, output_data, batch_size)
  train_step(x_batch, y_batch)

  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=iter)

  file_select_test()
  x_batch, y_batch = get_batch(accelometer, gyroscope,speed, output_data, batch_size)
  loss = test_step(x_batch, y_batch)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss_test', test_loss.result(), step=iter)

  # Update the model with the changed weights!
  if iter % 100 == 0:     
    model.save_weights(checkpoint_prefix)

  template = 'iter {}, Loss: {}, Test Loss: {}'
  print (template.format(iter+1,
                         train_loss.result(), 
                         test_loss.result())) 


  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

7, Test Loss: 103.97442626953125
iter 1699, Loss: 0.6436759233474731, Test Loss: 22.13088035583496
iter 1700, Loss: 0.5053232312202454, Test Loss: 66.53489685058594
iter 1701, Loss: 0.12565214931964874, Test Loss: 94.4637680053711
iter 1702, Loss: 0.5425272583961487, Test Loss: 60.1038818359375
iter 1703, Loss: 0.7526385188102722, Test Loss: 32.712867736816406
iter 1704, Loss: 0.09414172172546387, Test Loss: 44.25996780395508
iter 1705, Loss: 0.6299312710762024, Test Loss: 35.78813171386719
iter 1706, Loss: 0.4997311532497406, Test Loss: 381.4947814941406
iter 1707, Loss: 0.16777320206165314, Test Loss: 43.22249984741211
iter 1708, Loss: 0.8313570022583008, Test Loss: 457.0963134765625
iter 1709, Loss: 0.5930596590042114, Test Loss: 2665.647216796875
iter 1710, Loss: 0.4999663829803467, Test Loss: 7716.30615234375
iter 1711, Loss: 0.4959437847137451, Test Loss: 1103.661865234375
iter 1712, Loss: 0.6611565947532654, Test Loss: 63.22999954223633
iter 1713, Loss: 0.9166948795318604, Test 

In [354]:
%tensorboard --logdir /Users/amitaflalo/Desktop/deepnav/logs


Reusing TensorBoard on port 6006 (pid 3953), started 0:52:10 ago. (Use '!kill 3953' to kill it.)